# The Variance Estimator
Let $P_\mathbf{x} := \mathbb{P}(\mathbf{z} = \mathbf{x})$.
Let $\mathbf{z}^*$ be the realized treatment vector.
Let $P^* := \mathbb{P}(\mathbf{z} = \mathbf{z^*})$ and $P_S^* := \mathbb{P}(\mathbf{z_S} = \mathbf{z_S^*})$ for $S\subseteq[n]$.
Then, the variance estimator is given by $\widehat{\text{Var}}(\widehat{TTE}) = (*) + (**)$ where 
$$(*) = \frac{1}{n^2} \sum_{i=1}^n \sum_{j \in M_i} \frac{1}{P_{N_i \cup N_j}^*} \cdot Y_i(\mathbf{z}^*)w_i(\mathbf{z}^*) \cdot Y_j(\mathbf{z}^*)w_j(\mathbf{z}^*)\cdot \text{Cov}_{ij}$$
and
$$(**) = \frac{1}{n^2} \sum_{i=1}^n P_{N_i}^* \cdot Y_i^2\mathbf{z^*} w_i^2\mathbf{z^*} \cdot \sum_{j \in M_i} 2^{|N_j|} - 2^{|N_j\setminus N_i|}$$
with 
$$\text{Cov}_{ij} := \prod_{k \in N_i \cup N_j} p^{\mathbf{z}_k^*}(1-p)^{1-\mathbf{z}_k^*}\left(1 - \prod_{\ell \in N_i \cap N_j} p^{\mathbf{z}_\ell^*}(1-p)^{1-\mathbf{z}_\ell^*}\right).$$

## Computing Term 1
We compute the first term, $(*)$, as follows:
- First, create an $n \times 1$ array $\mathbf{YW}$ with entry $i$ equal to $Y_i(\mathbf{z}^*)w_i(\mathbf{z}^*)$.
- Next, we create an $n \times 1$ array $\mathbf{V}$ with entry $i$ equal to $$\sum_{j \in M_i} \frac{1}{P_{N_i \cup N_j}^*} \cdot Y_j(\mathbf{z}^*)w_j(\mathbf{z}^*)\cdot \text{Cov}_{ij}.$$
    -- For each $i \in [n]$, we create the following $m_i := |M_i| \times 1$ arrays: 
    $$\mathbf{P}_i := \begin{bmatrix} 1/P_{N_i \cup N_{j_1}} \\ \vdots \\ 1/P_{N_i \cup N_{j_{m_i}}} \end{bmatrix},\ 
    \mathbf{YW}_i = \begin{bmatrix} Y_{j_1}(\mathbf{z}^*)w_{j_1}(\mathbf{z}^*) \\ \vdots \\ Y_{j_{m_i}}(\mathbf{z}^*)w_{j_{m_i}}(\mathbf{z}^*) \end{bmatrix}, \
    \textbf{COV}_i := \begin{bmatrix} \text{Cov}_{i{j_1}} \\ \vdots \\ \text{Cov}_{i{j_{m_i}}} \end{bmatrix}.$$

    -- Then, entry-wise multiply these arrays to get one array, sum the entries of the resulting array, and that sum is the $i$-th entry of $\mathbf{V}$.
- Then, return the dot product of $\mathbf{YW}$ and $\mathbf{V}$ divided by $n^2$.

## Computing Term 2
We compute the second term, $(**)$, as follows:
- First, compute an $n \times 1$ array $\mathbf{PY2W2}$ where entry $i$ equals $P_{N_i}^* Y_i^2(\mathbf{z}^*) w_i^2(\mathbf{z}^*).$
- Then, compute an $n \times 1$ array $\mathbf{CNTS}$ where entry $i$ equals $$\sum_{j\in M_i} 2^{|N_j|} - 2^{|N_j \setminus N_i|}.$$
- Finally, take the dot product of $\mathbf{PY2W2}$ and $\mathbf{CNTS}$ and divide by $n^2$

## The Code

In [1]:
import numpy as np
import scipy.sparse

In [2]:
def var_est(n, p, y, A, z):
    '''
    n : int
        size of the population
    p : float
        treatment probability
    y : numpy array
        observations
    A : numpy array 
        adjacency matrix where (i,j)th entry = 1 iff j's treatment affects i's outcome
    z : numpy array
        realized treatment assignment vector
    '''
    zz = z/p - (1-z)/(1-p)
    w = A.dot(zz)
    YW = y * w
    YW_sq = np.square(YW)

    V = np.zeros(n)
    PY2W2 = np.zeros(n)
    CNTS = np.zeros(n)

    prob_p = np.power(np.ones(n)*p, z) 
    prob_1_minus_p = np.power(1 - np.ones(n)*p, 1 - z)

    dep_neighbors = np.dot(A,A.T)
    
    for i in np.arange(n):
        Ni = np.nonzero(A[[i],:])
        #print("Ni: {}".format(Ni))
        #print(Ni[1])
        Mi = np.nonzero(dep_neighbors[[i],:]) # dependency neighbor's indices
        Mi = Mi[1]
        #print("Mi: {}".format(Mi))
        #print(Mi[1])

        Pi = np.zeros(len(Mi))
        COVi = np.zeros(len(Mi))
        sum = 0
        for j in np.arange(len(Mi)):
            Nj = np.nonzero(A[[Mi[j]], :])
            Ni_or_Nj = np.union1d(Ni,Nj)

            # Compute Pi
            mult_p = prob_p[Ni_or_Nj]
            mult_1_minus_p = prob_1_minus_p[Ni_or_Nj]
            Pi[j] = np.prod(mult_p) * np.prod(mult_1_minus_p)

            # Compute COVi
            Ni_and_Nj = np.intersect1d(Ni,Nj)
            mult_p = prob_p[Ni_and_Nj]
            mult_1_minus_p = prob_1_minus_p[Ni_and_Nj]
            temp = np.prod(mult_p) * np.prod(mult_1_minus_p)
            COVi[j] = Pi[j] * (1 - temp)

            # Compute CNTS[i]
            Nj_minus_Ni = np.setdiff1d(Nj,Ni)
            sum = sum + (2**len(Nj) - 2**len(Nj_minus_Ni))


        # Compute V
        V[i] = np.sum(1/Pi * YW[Mi] * COVi)

        # Compute PY2W2
        mult_p = prob_p[Ni[1]]
        mult_1_minus_p = prob_1_minus_p[Ni[1]]
        PY2W2[i] = np.prod(mult_p) * np.prod(mult_1_minus_p) * YW_sq[i]

        # Compute CNTS
        CNTS[i] = sum
    
    term1 = (1/n**2) * np.dot(YW, V)
    term2 = (1/n**2) * np.dot(PY2W2, CNTS)
    return term1+term2



### Experiments

In [3]:
def var_bound(n, p, A, C, alp, beta=1):
    '''
    Returns the conservative upper bound on the variance of the SNIPE(beta) estimator

    n (int): size of the population
    p (float): treatment probability
    A (scipy sparse array): adjacency matrix
    C (scipy sparse array): weighted adjacency matrix
    alp (numpy array): baseline effects
    beta (int): degree of the potential outcomes model
    '''
    in_deg = scipy.sparse.diags(np.array(A.sum(axis=1)).flatten(),0)  # array of the in-degree of each node
    out_deg = scipy.sparse.diags(np.array(A.sum(axis=0)).flatten(),0)  # array of the out-degree of each node
    in_deg = in_deg.tocsr() 
    out_deg = out_deg.tocsr() 

    d_in = in_deg.max()
    d_out = out_deg.max()
    temp = max(4 * (beta**2), (1 / (p*(1-p))))

    if beta == 1:
        Ymax = np.amax(scipy.sparse.diags(np.array(C.sum(axis=1)).flatten(),0) + alp)
    else:
        Ymax = np.amax(1 + alp)

    bound = (1/n) * d_in * d_out * (Ymax**2) * (np.exp(1) * d_in * temp)**beta * (1/beta)**beta
    return bound

In [4]:
linear_pom = lambda C,alpha, z : C.dot(z) + alpha
bernoulli = lambda n,p : (np.random.rand(n) < p) + 0

def erdos_renyi(n,p):
    '''
    Generates a random network of n nodes using the Erdos-Renyi method,
    where the probability that an edge exists between two nodes is p.

    Returns the adjacency matrix of the network as an n by n numpy array
    '''
    A = np.random.rand(n,n)
    A = (A < p) + 0
    A[range(n),range(n)] = 1   # everyone is affected by their own treatment
    return scipy.sparse.csr_array(A)

def simpleWeights(A, diag=5, offdiag=5, rand_diag=np.array([]), rand_offdiag=np.array([])):
    '''
    Returns weights generated from simpler model

    A (numpy array): adjacency matrix of the network
    diag (float): maximum norm of direct effects
    offidiag (float): maximum norm of the indirect effects
    '''
    n = A.shape[0]

    if rand_offdiag.size == 0:
        rand_offdiag = np.random.rand(n)
    C_offdiag = offdiag*rand_offdiag

    in_deg = scipy.sparse.diags(np.array(A.sum(axis=1)).flatten(),0)  # array of the in-degree of each node
    C = in_deg.dot(A - scipy.sparse.eye(n))
    col_sum = np.array(C.sum(axis=0)).flatten()
    col_sum[col_sum==0] = 1
    temp = scipy.sparse.diags(C_offdiag/col_sum)
    C = C.dot(temp)

    # out_deg = np.array(A.sum(axis=0)).flatten() # array of the out-degree of each node
    # out_deg[out_deg==0] = 1
    # temp = scipy.sparse.diags(C_offdiag/out_deg)
    # C = A.dot(temp)

    if rand_diag.size == 0:
        rand_diag = np.random.rand(n)
    C_diag = diag*rand_diag
    C.setdiag(C_diag)

    return C

def est_us(n, p, y, A, z):
    '''
    Returns an estimate of the TTE using our proposed estimator

    n (int): number of individuals
    p (float): treatment probability
    y (numpy array?): observations
    A (square numpy array): network adjacency matrix
    z (numpy array): treatment vector
    '''
    zz = z/p - (1-z)/(1-p)
    return 1/n * y.dot(A.dot(zz))

In [5]:
n = 5000
r = 2
diag = 1
offdiag = r*diag
p = 0.05

# Create Weighted Adjcency matrix
deg = 10
A = erdos_renyi(n,deg/n)
rand_wts = np.random.rand(n,3)
alpha = rand_wts[:,0].flatten() # baseline effects
C = simpleWeights(A, diag, offdiag, rand_wts[:,1].flatten(), rand_wts[:,2].flatten())

# Potential outcomes function
fy = lambda z: linear_pom(C,alpha,z)

TTE = 1/n * np.sum((fy(np.ones(n)) - fy(np.zeros(n))))
print("Ground-Truth TTE: {}\n".format(TTE))

Ground-Truth TTE: 1.5079827930890148



/Users/mayleencortez/Desktop/NetworkCausalInference/ClusterRD/low-order-clusterRD/.venv/lib/python3.11/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [6]:
T = 150
TTE_hat, TTE_var_hat = np.zeros(T), np.zeros(T)

for i in range(T):
    z = bernoulli(n,p)
    y = fy(z)

    TTE_hat[i] = est_us(n, p, y, A, z)
    TTE_var_hat[i] = var_est(n, p, y, A, z)

bound = var_bound(n, p, A, C, alpha)

print("SNIPE: {}".format(np.sum(TTE_hat)/T))
print("SNIPE bias: {}\n".format(((np.sum(TTE_hat)/T) - TTE)/TTE))

exp_var = np.sum((TTE_hat-TTE)**2)/T
print("MSE (Experimental Variance): {}".format(exp_var))
print("Variance Bound: {}".format(bound))
print("Variance Estimate: {}\n".format(np.sum(TTE_var_hat)/T))
print("Variance Estimator bias: {}\n".format(((np.sum(TTE_var_hat)/T) - exp_var)))


SNIPE: 1.5962803291657306
SNIPE bias: 0.05855341087536112

MSE (Experimental Variance): 0.20949495075876856
Variance Bound: 7523.157657577504
Variance Estimate: -2.0807634612964176

Variance Estimator bias: -2.290258412055186

